In [8]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# Данный файл содержит алгоритмы построения графиков различных зависимостей

In [9]:
from lrnn import (
    LRNN,
    image_to_blocks,
    compress_image,
    decompress_image
)
import numpy as np
from PIL import Image
import plotly.graph_objects as go

In [10]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [11]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('car.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.1))])


network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=2000)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 4.807507403561401
Epoch 1/15000, Loss: 2346.1969900596027
Epoch 2/15000, Loss: 2279.9013389756874
Epoch 3/15000, Loss: 2238.364867841011
Epoch 4/15000, Loss: 2208.69137833023
Epoch 5/15000, Loss: 2185.8639293934466
Epoch 6/15000, Loss: 2165.3937603791132
Epoch 7/15000, Loss: 2148.2767331560294
Epoch 8/15000, Loss: 2133.485170379791
Epoch 9/15000, Loss: 2113.87567578737
Epoch 10/15000, Loss: 2091.2840988332787
Epoch 11/15000, Loss: 2075.168784916295
Epoch 12/15000, Loss: 2052.504444737696
Epoch 13/15000, Loss: 2033.450200826287
Epoch 14/15000, Loss: 2012.043607933841
Epoch 15/15000, Loss: 1992.5246395773556
Compression coefficient: 2.466780686751743
Z = 4.807507403561401


## Relation between learning rate and epochs 

In [12]:
alphas = (0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001)
epochs_amount: list[int] = []
for alpha in alphas:
    dl = LRNN(n, p, alpha)
    dl.train(train, 15000, learn_by_loss=True, max_loss=2500)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Коэффициент обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 7676.470724395473
Epoch 2/15000, Loss: 5393.370306840883
Epoch 3/15000, Loss: 4408.508344592167
Epoch 4/15000, Loss: 3838.1538306442776
Epoch 5/15000, Loss: 3482.82505157057
Epoch 6/15000, Loss: 3231.4485157334257
Epoch 7/15000, Loss: 3054.4094224293594
Epoch 8/15000, Loss: 2923.7475761500423
Epoch 9/15000, Loss: 2822.9232899994468
Epoch 10/15000, Loss: 2740.4085204027983
Epoch 11/15000, Loss: 2671.4887780832814
Epoch 12/15000, Loss: 2613.7277274697785
Epoch 13/15000, Loss: 2562.6383709037013
Epoch 14/15000, Loss: 2517.6657453578723
Epoch 15/15000, Loss: 2477.1223872145965
Epoch 1/15000, Loss: 7881.673972124617
Epoch 2/15000, Loss: 5442.669498266883
Epoch 3/15000, Loss: 4359.490447690583
Epoch 4/15000, Loss: 3753.181958989009
Epoch 5/15000, Loss: 3378.2374500960723
Epoch 6/15000, Loss: 3131.3702116238655
Epoch 7/15000, Loss: 2962.167027978666
Epoch 8/15000, Loss: 2834.455997655034
Epoch 9/15000, Loss: 2734.696564212712
Epoch 10/15000, Loss: 2651.9352302643492
Epoch

## Relationship between hidden layer dimension and epochs

In [13]:
p_s = (10, 15, 20, 25, 30, 35, 40)
epochs_amount: list[int] = []
compression_rates: list[float] = []
for p in p_s:    
    dl = LRNN(n, p, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=2000)
    epochs_amount.append(dl.epoch)

    compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
    compression_info_size = (
        compressed.size * compressed.itemsize +
        network.W_dec.size * network.W_dec.itemsize +
        np.array(shape).size * np.array(shape).itemsize +
        np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
    ) * 8
    compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)

fig = go.Figure()
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Коэффициент сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 4491.526254909077
Epoch 2/15000, Loss: 3302.794441415094
Epoch 3/15000, Loss: 2904.613654337254
Epoch 4/15000, Loss: 2678.149683038423
Epoch 5/15000, Loss: 2532.8816365525604
Epoch 6/15000, Loss: 2429.1547492046852
Epoch 7/15000, Loss: 2356.3151619541895
Epoch 8/15000, Loss: 2302.392259882621
Epoch 9/15000, Loss: 2262.3727133344028
Epoch 10/15000, Loss: 2229.807518955636
Epoch 11/15000, Loss: 2204.5316837035907
Epoch 12/15000, Loss: 2182.935393667307
Epoch 13/15000, Loss: 2164.044198089747
Epoch 14/15000, Loss: 2146.177941995376
Epoch 15/15000, Loss: 2131.1608565225465
Epoch 16/15000, Loss: 2117.6129978526465
Epoch 17/15000, Loss: 2104.40426030155
Epoch 18/15000, Loss: 2091.484072575025
Epoch 19/15000, Loss: 2080.684839506644
Epoch 20/15000, Loss: 2068.9405954077383
Epoch 21/15000, Loss: 2058.323200633382
Epoch 22/15000, Loss: 2048.2835072079447
Epoch 23/15000, Loss: 2037.8947610905761
Epoch 24/15000, Loss: 2028.147291928682
Epoch 25/15000, Loss: 2018.31598200863
E

## Relationship between max error and epochs

In [14]:
max_errors = list(range(1000, 3000, 100))
epochs_amount: list[int] = []
for err in max_errors:    
    dl = LRNN(n, 50, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=err)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 2867.8380828076415
Epoch 2/15000, Loss: 2289.502263030319
Epoch 3/15000, Loss: 1975.5891173042714
Epoch 4/15000, Loss: 1804.1068999608096
Epoch 5/15000, Loss: 1699.0479739448203
Epoch 6/15000, Loss: 1623.9196498321126
Epoch 7/15000, Loss: 1564.0939516573742
Epoch 8/15000, Loss: 1514.2802400844494
Epoch 9/15000, Loss: 1471.7240788851313
Epoch 10/15000, Loss: 1433.8118070031958
Epoch 11/15000, Loss: 1397.508254715833
Epoch 12/15000, Loss: 1367.461497015521
Epoch 13/15000, Loss: 1338.8185551481256
Epoch 14/15000, Loss: 1312.1719795382905
Epoch 15/15000, Loss: 1286.9899783491173
Epoch 16/15000, Loss: 1263.837373836849
Epoch 17/15000, Loss: 1242.6653576417036
Epoch 18/15000, Loss: 1222.4731744745889
Epoch 19/15000, Loss: 1203.4122310957914
Epoch 20/15000, Loss: 1184.63517259464
Epoch 21/15000, Loss: 1166.8133878000526
Epoch 22/15000, Loss: 1149.7586516504089
Epoch 23/15000, Loss: 1134.588437014518
Epoch 24/15000, Loss: 1119.2928409768238
Epoch 25/15000, Loss: 1105.17292

## Relationship between image and epochs

In [15]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 50 

img = Image.open('images/winter.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1500)

print(network.epoch)

# compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg_array = np.asarray(dimg)
# dimg.save('compression-decompression_test.jpg')

Z = 1.9230473377332657
Epoch 1/15000, Loss: 7516.0534758628855
Epoch 2/15000, Loss: 4447.567689083428
Epoch 3/15000, Loss: 3527.883201262848
Epoch 4/15000, Loss: 3057.702406077855
Epoch 5/15000, Loss: 2762.2270547929083
Epoch 6/15000, Loss: 2553.2287967491984
Epoch 7/15000, Loss: 2391.7273205824504
Epoch 8/15000, Loss: 2261.8527300554156
Epoch 9/15000, Loss: 2152.3316969925568
Epoch 10/15000, Loss: 2058.9453200917433
Epoch 11/15000, Loss: 1978.7882829499738
Epoch 12/15000, Loss: 1907.6307963106685
Epoch 13/15000, Loss: 1843.8665703742442
Epoch 14/15000, Loss: 1787.0528248755738
Epoch 15/15000, Loss: 1736.190013422143
Epoch 16/15000, Loss: 1688.9232111100512
Epoch 17/15000, Loss: 1645.4353120509074
Epoch 18/15000, Loss: 1605.9185154393401
Epoch 19/15000, Loss: 1569.5819992500262
Epoch 20/15000, Loss: 1536.062310195727
Epoch 21/15000, Loss: 1505.0931899921154
Epoch 22/15000, Loss: 1475.699896237704
22
